In [1]:
import re
import pandas as pd

In [ ]:
f = open("WhatsApp Chat.txt","r",encoding='utf-8')

In [3]:
data = f.read()

In [ ]:
print(data)

In [4]:
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [ ]:
messages = re.split(pattern,data)[1:]
messages

In [ ]:
dates = re.findall(pattern,data)
dates

In [ ]:
df = pd.DataFrame({'user_message':messages,"message_date":dates})

df['message_date'] = pd.to_datetime(df['message_date'].str.rstrip(), format='%d/%m/%y, %H:%M -')


df.rename(columns={'message_date':'date'},inplace=True)

df.head()

In [8]:
df.shape

(10095, 2)

In [9]:
users = []
messages = []
for message in df['user_message']:
  entry = re.split('([\w\W]+?):\s',message)
  if entry[1:]:
    users.append(entry[1])
    messages.append(entry[2])
  else:
    users.append('group_notification')
    messages.append(entry[0])

df['user'] = users
df ['message'] = messages
df.drop(columns=['user_message'],inplace=True)


In [ ]:
df.head()

In [10]:
df['year'] = df['date'].dt.year


In [11]:
df['month'] = df['date'].dt.month_name()

In [12]:
df['day'] = df['date'].dt.day

In [13]:
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

In [ ]:
# Import TextBlob for sentiment analysis
from textblob import TextBlob

# Perform sentiment analysis and add a sentiment column to the DataFrame
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['message'].apply(get_sentiment)

# Map sentiment labels to numeric values
sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}
df['sentiment_numeric'] = df['sentiment'].map(sentiment_mapping)

# Display the DataFrame with the new sentiment column
print(df.head())


In [ ]:
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("")

# Concatenate all messages into a single string
all_messages = "\n".join(df['message'])


# Split input text into chunks
chunk_size = 1000  # You can adjust this value based on the maximum input size supported by the model
chunks = [all_messages[i:i+chunk_size] for i in range(0, len(all_messages), chunk_size)]

# Set maximum length of the summary
max_length = 100

# Generate summaries for each chunk
summaries = []
for chunk in chunks:
    summary = summarizer(chunk, max_length=max_length, min_length=30, do_sample=False)
    summaries.append(summary[0]['summary_text'])

print("Summary:", summary[0]['summary_text'])


In [15]:
f = open('stop_hinglish.txt','r')
stop_words = f.read()

In [ ]:
# Combine messages by user and date
combined_messages = df.groupby(['user', 'date'])['message'].apply(lambda x: ' '.join(x)).reset_index()
combined_messages

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install demoji

In [29]:
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()

def preprocess_text(text):
    # Tokenize text into words
    words = word_tokenize(text)
    # Remove unnecessary characters and convert to lowercase
    cleaned_words = [' '.join(word.lower().split()) for word in words if word.isalnum()]
    # Perform stemming and remove stop words
    cleaned_words = [ps.stem(word) for word in cleaned_words if word not in stop_words]
    return ' '.join(cleaned_words)

combined_messages['processed_message'] = combined_messages['message'].apply(preprocess_text)
# Tokenize the text into sentences
sentences = combined_messages['processed_message'].apply(sent_tokenize)

import demoji

# Download emoji data (only need to run once)
demoji.download_codes()

# Function to remove emojis from text
def remove_emojis(text):
    return demoji.replace(text, '')

# Apply emoji removal to the 'processed_message' column
combined_messages['processed_message'] = combined_messages['processed_message'].apply(remove_emojis)

# Format data for text summarization
documents = combined_messages['processed_message'].tolist()


<ipython-input-29-43077cd983d2>:23: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
documents

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)


In [32]:
tfidf_vectorizer.get_feature_names_out()

array(['0023', '0116', '013', ..., 'पहन', 'भवन', 'वह'], dtype=object)

In [33]:
tfidf_matrix.shape

(4184, 3585)

In [36]:
sentences = sentences.tolist()

In [50]:
from sklearn.cluster import KMeans
import numpy as np
# Use KMeans clustering to identify representative sentences
n_clusters = 5  # Number of clusters (can be adjusted)
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(tfidf_matrix)
cluster_centers = kmeans.cluster_centers_
cluster_labels = kmeans.labels_

# Find sentences closest to cluster centers
representative_sentences = []
for i in range(n_clusters):
    cluster_indices = np.where(cluster_labels == i)[0]
    centroid_distances = np.linalg.norm(tfidf_matrix[cluster_indices] - cluster_centers[i], axis=1)
    representative_idx = cluster_indices[np.argmin(centroid_distances)]
    representative_sentences.append(documents[representative_idx])

# Print representative sentences for each cluster
for i, sentences in enumerate(representative_sentences):
    print(f"Cluster {i+1}:")
    for sent in sentences:
        print(sent)
    print()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 1:
s
e
e
d
h
a

Cluster 2:
w
a
i
t
 
m
e
s
s
a
g

Cluster 3:
m
e
s
s
a
g
 
d
e
l
e
t

Cluster 4:

Cluster 5:
m
e
d
i
a
 
o
m
i
t



In [67]:
cluster_labels

array([1, 1, 1, ..., 3, 3, 3], dtype=int32)

In [72]:
def generate_summary(document):

    # Tokenize the document into sentences
    sentences = sent_tokenize(document)

    # Clean and preprocess the sentences (similar to the preprocessing steps used during training)
    cleaned_sentences = []
    for sent in sentences:
        words = word_tokenize(sent.lower())
        # Remove unnecessary characters and convert to lowercase
        cleaned_words = [' '.join(word.lower().split()) for word in words if word.isalnum()]
        # Perform stemming and remove stop words
        cleaned_words = [ps.stem(word) for word in cleaned_words if word not in stop_words]
        cleaned_sentences.append(' '.join(cleaned_words))

    # Convert cleaned sentences into TF-IDF matrix
    tfidf_matrix = tfidf_vectorizer.transform(cleaned_sentences)

    if tfidf_matrix.shape[0] == 0:
        raise ValueError("TF-IDF matrix is empty")
    # Predict cluster labels for the sentences using the trained KMeans model
    cluster_labels = kmeans.predict(tfidf_matrix)
    print(cluster_labels)
    # Select representative sentences from each cluster
    representative_sentences = []
    for i in range(3,4):
        print(i)
        cluster_indices = np.where(cluster_labels == i)
        print(cluster_indices)
        centroid_distances = np.linalg.norm(tfidf_matrix[cluster_indices] - cluster_centers[i], axis=1)
        representative_idx = cluster_indices[np.argmin(centroid_distances)]
        representative_sentences.append(sentences[representative_idx])

    # Concatenate representative sentences to form the summary
    summary = ' '.join(representative_sentences)

    return summary


In [ ]:
# Sample document
document = """
Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.

NLP tasks include text translation, sentiment analysis, speech recognition, and text summarization. Text summarization aims to condense a piece of text while preserving its key information and meaning.

There are two main approaches to text summarization: extractive and abstractive. Extractive summarization involves selecting and concatenating important sentences or phrases from the original text. Abstractive summarization involves generating new sentences that capture the essence of the original text.

In this example, we will use an extractive summarization approach based on KMeans clustering. We will tokenize the text, compute TF-IDF vectors for the sentences, and use KMeans to cluster the sentences into representative groups. Then, we will select one representative sentence from each cluster to form the summary.
"""

# Test generate_summary function
summary = generate_summary(document)

# Print generated summary
print("Generated Summary:")
print(summary)
